In [2]:
from konlpy.tag import Mecab 

mec = Mecab(dicpath=r"C:\mecab\mecab-ko-dic")

In [10]:
import pandas as pd

df = pd.read_csv("df3_2_artsCleanEtc.csv")

df_content_total = df
# # sample
# df_sample= df.loc[:1000]


In [3]:
mec.pos('한가요')

[('한', 'MM'), ('가요', 'NNG')]

In [18]:

#초기 구조화: 필수 -> 이후 case로 분기

series_mec = df_content_total['art_content'].map(lambda x : mec.pos(x, flatten=False, join=False))

df_mec = pd.DataFrame(columns=['art_id','token'])

for i in range(len(series_mec)):
    print(f'level1: {i} =======================')
    for j in range(len(series_mec[i])):
        df_mec = df_mec.append({'art_id':i, 'token': series_mec[i][j]}, ignore_index=True)
        print(f'level2: {j}')
        
# df_mec.head(len(df_mec))     
df_mec.to_csv('df_mec_content_all.csv',encoding='utf-8-sig', index=False)


level1: 0 =======================
level2: 0
level2: 1
level2: 2
level2: 3
level2: 4
level2: 5
level2: 6
level2: 7
level2: 8
level2: 9
level2: 10
level2: 11
level2: 12
level2: 13
level2: 14
level2: 15
level2: 16
level2: 17
level2: 18
level2: 19
level2: 20
level2: 21
level2: 22
level2: 23
level2: 24
level2: 25
level2: 26
level2: 27
level2: 28
level2: 29
level2: 30
level2: 31
level1: 1 =======================
level2: 0
level2: 1
level2: 2
level2: 3
level2: 4
level2: 5
level2: 6
level2: 7
level2: 8
level2: 9
level2: 10
level2: 11
level2: 12
level2: 13
level2: 14
level2: 15
level2: 16
level2: 17
level2: 18
level2: 19
level2: 20
level2: 21
level2: 22
level2: 23
level2: 24
level2: 25
level2: 26
level2: 27
level2: 28
level2: 29
level2: 30
level2: 31
level2: 32
level2: 33
level2: 34
level2: 35
level2: 36
level2: 37
level2: 38
level2: 39
level2: 40
level2: 41
level2: 42
level2: 43
level2: 44
level2: 45
level2: 46
level2: 47
level2: 48
level2: 49
level2: 50
level2: 51
level2: 52
level2: 53
level2

KeyboardInterrupt: 

In [5]:
# (CASE1) 복합명사 처리x 기본 형태소 단위 그대로 넣은 경우 비교용

df_res_basic = pd.DataFrame(columns=['art_id','token','type'])

for i in range(len(df_mec)):
    for j in range(len(df_mec.loc[i][1])):
        df_res_basic = df_res_basic.append({'art_id':df_mec.loc[i][0], 'token':df_mec.loc[i][1][j][0], 'type':df_mec.loc[i][1][j][1]},ignore_index=True)
               
print(df_res_basic)

# df_res_basic.to_csv('df_res_basic.csv', encoding='utf-8-sig', index=False)

     art_id token    type
0         0   erp      SL
1         0  프로그램     NNG
2         0    오류     NNG
3         0    장애     NNG
4         0    조치     NNG
...     ...   ...     ...
1076    100     는      JX
1077    100   MAC      SL
1078    100     이     JKS
1079    100    정답     NNG
1080    100   일까요  VCP+EC

[1081 rows x 3 columns]


In [4]:
# (CASE4) mecab 배열상 type 연속성 기반 복합명사화 : NNG,NNP-NNG,NNP 혼재 CASE 
    # 문제: 3개 연달아 등장하는 명사 해결법
    # 문제: 연

df_res_NNG_NNP = pd.DataFrame(columns=['art_id','token','type'])
# df_res_NNG_NNP.append({'art_id':, 'token','type'})

for i in range(len(df_mec)):
    if len(df_mec.loc[i][1]) <= 1 :
        df_res_NNG_NNP = df_res_NNG_NNP.append({'art_id':df_mec.loc[i][0],'token':df_mec.loc[i][1][0][0],'type':df_mec.loc[i][1][0][1]},ignore_index=True)
        continue
    for j in range(1,len(df_mec.loc[i][1])):
        if (df_mec.loc[i][1][j-1][1] == 'NNG' or df_mec.loc[i][1][j-1][1] == 'NNP') and (df_mec.loc[i][1][j][1] == 'NNG' or df_mec.loc[i][1][j][1] == 'NNP') :
            df_res_NNG_NNP = df_res_NNG_NNP.append({'art_id':df_mec.loc[i][0],'token': "".join([df_mec.loc[i][1][j-1][0], df_mec.loc[i][1][j][0] ]) ,'type':'comp_NNG_NNP'},ignore_index=True)
            continue
        df_res_NNG_NNP = df_res_NNG_NNP.append({'art_id':df_mec.loc[i][0], 'token':df_mec.loc[i][1][j][0] ,'type':df_mec.loc[i][1][j][1]}, ignore_index=True)


df_res_NNG_NNP.to_csv('df_res_content_NNG_NNP_all.csv', encoding='utf-8-sig', index=False)

print(df_res_NNG_NNP)

     art_id token          type
0         0   erp            SL
1         0  프로그램           NNG
2         0    오류           NNG
3         0  장애조치  comp_NNG_NNP
4         0    방법           NNG
...     ...   ...           ...
6578   1000    고수           NNP
6579   1000     님           XSN
6580   1000     들           XSN
6581   1000     주            VX
6582   1000    세요         EP+EF

[6583 rows x 3 columns]
